# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import seaborn as sns

# Import API key
from api_keys import geoapify_key

In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities_updated.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,alice springs,-23.7000,133.8833,54.95,58,40,16.11,AU,1717640999
1,1,wailua homesteads,22.0669,-159.3780,86.43,79,40,10.36,US,1717641007
2,2,grytviken,-54.2811,-36.5092,28.15,96,100,5.03,GS,1717641008
3,3,puerto ayora,-0.7393,-90.3518,71.55,96,36,9.48,EC,1717641009
4,4,lompoc,34.6391,-120.4579,76.01,85,0,3.44,US,1717641010


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 85) & (city_data_df["Max Temp"] > 65) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
narrowed_city_df = narrowed_city_df.dropna()

# Display sample data
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,lompoc,34.6391,-120.4579,76.01,85,0,3.44,US,1717641010
25,25,gracemere,-23.4333,150.4500,77.38,38,0,3.44,AU,1717641024
36,36,villa yapacani,-17.4000,-63.8333,66.24,75,0,2.66,BO,1717641032
115,115,gori,41.9842,44.1158,65.35,73,0,1.16,GE,1717641084
144,144,east ballina,-28.8667,153.5833,68.79,55,0,3.78,AU,1717641102
263,263,tsiombe,-25.3000,45.4833,66.94,81,0,3.85,MG,1717641178
274,274,astrakhan,46.3497,48.0408,66.07,94,0,2.24,RU,1717641186
294,294,ar rutbah,33.0381,40.2848,83.25,18,0,4.32,IQ,1717641200
337,337,diamantino,-14.4086,-56.4461,69.53,54,0,4.16,BR,1717641228
344,344,san felice circeo,41.2372,13.0942,68.58,72,0,3.06,IT,1717641232


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
4,lompoc,US,34.6391,-120.4579,85,
25,gracemere,AU,-23.4333,150.4500,38,
36,villa yapacani,BO,-17.4000,-63.8333,75,
115,gori,GE,41.9842,44.1158,73,
144,east ballina,AU,-28.8667,153.5833,55,
263,tsiombe,MG,-25.3000,45.4833,81,
274,astrakhan,RU,46.3497,48.0408,94,
294,ar rutbah,IQ,33.0381,40.2848,18,
337,diamantino,BR,-14.4086,-56.4461,54,
344,san felice circeo,IT,41.2372,13.0942,72,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
lompoc - nearest hotel: Red Roof Inn Lompoc
gracemere - nearest hotel: Rockhampton Plaza
villa yapacani - nearest hotel: No hotel found
gori - nearest hotel: ინტურისტი
east ballina - nearest hotel: Ballina Beach Resort
tsiombe - nearest hotel: No hotel found
astrakhan - nearest hotel: Omega
ar rutbah - nearest hotel: No hotel found
diamantino - nearest hotel: Hotel kayaby
san felice circeo - nearest hotel: Centro Ferie Salvatore
urumqi - nearest hotel: 百时快捷酒店红山店
jeddah - nearest hotel: No hotel found
shihezi - nearest hotel: 石河子宾馆
dogonbadan - nearest hotel: هتل پاسارگاد
nsanje - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
4,lompoc,US,34.6391,-120.4579,85,Red Roof Inn Lompoc
25,gracemere,AU,-23.4333,150.4500,38,Rockhampton Plaza
36,villa yapacani,BO,-17.4000,-63.8333,75,No hotel found
115,gori,GE,41.9842,44.1158,73,ინტურისტი
144,east ballina,AU,-28.8667,153.5833,55,Ballina Beach Resort
263,tsiombe,MG,-25.3000,45.4833,81,No hotel found
274,astrakhan,RU,46.3497,48.0408,94,Omega
294,ar rutbah,IQ,33.0381,40.2848,18,No hotel found
337,diamantino,BR,-14.4086,-56.4461,54,Hotel kayaby
344,san felice circeo,IT,41.2372,13.0942,72,Centro Ferie Salvatore


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [14]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)